In [23]:
import os
import json
import pandas as pd
import requests
import time
import datetime
from datetime import date
import calendar
import csv

In [24]:
# date

after_start_date = '2016-04'
before_end_date = '2019-04'
# before_end_date = '2016-05'

def get_start_end_day(year, month):
    _, monthRange = calendar.monthrange(year, month)
    first_day = datetime.date(year=year, month=month, day=1)
    last_day = datetime.date(year=year, month=month, day=monthRange)
    return first_day, last_day

def get_each_month_in_a_list(after_start_date, before_end_date):
    month_pair_list = []
    start_year, start_month = after_start_date.split('-')
    end_year, end_month = before_end_date.split('-')
    start_year, start_month, end_year, end_month = int(start_year), int(start_month), int(end_year), int(end_month)
    
    temp_year, temp_month = start_year, start_month
    while(temp_year<=end_year):
        if(temp_year<end_year):
            while(temp_month<=12):
                first_day_of_month, last_day_of_month = get_start_end_day(temp_year, temp_month)
                month_pair_list.append((first_day_of_month, last_day_of_month))
                temp_month += 1
        else:
            while(temp_month<=end_month):
                first_day_of_month, last_day_of_month = get_start_end_day(temp_year, temp_month)
                month_pair_list.append((first_day_of_month, last_day_of_month))
                temp_month += 1
        temp_month = temp_month%12
        temp_year += 1
    return month_pair_list

def get_start_end_seconds(after_start_date, before_end_date):
    after_start_seconds = (after_start_date -  date.fromisoformat('1970-01-01')).total_seconds()
    before_end_seconds = (before_end_date -  date.fromisoformat('1970-01-01')).total_seconds()
    return after_start_seconds, before_end_seconds

def get_each_month_second_in_a_list(month_pair_list):
    month_second_pair_list = []
    for after_start_date, before_end_date in month_pair_list:
        after_start_seconds, before_end_seconds = get_start_end_seconds(after_start_date, before_end_date)
        month_second_pair_list.append((after_start_seconds, before_end_seconds))
    return month_second_pair_list
        
month_pair_list = get_each_month_in_a_list(after_start_date, before_end_date)
# print('month_pair_list: ', month_pair_list)
month_second_pair_list = get_each_month_second_in_a_list(month_pair_list)
# print('month_second_pair_list: ', month_second_pair_list)


# test mode
# after_start_date = date.fromisoformat('2016-04-01')
# before_end_date = date.fromisoformat('2016-04-30')
# after_start_seconds, before_end_seconds = get_start_end_seconds(after_start_date, before_end_date)
# print('after_start_seconds: ', after_start_seconds)
# print('before_end_seconds: ', before_end_seconds)

In [25]:
year_month_key_list = ['2016-04','2016-05','2016-06','2016-07','2016-08','2016-09','2016-10','2016-11','2016-12',
                       '2017-01','2017-02', '2017-03','2017-04','2017-05','2017-06','2017-07','2017-08','2017-09',
                       '2017-10','2017-11','2017-12', '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', 
                       '2018-06', '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12', '2019-01', 
                       '2019-02', '2019-03', '2019-04' ]
submission_year_month_dict = {}
for index in range(len(month_second_pair_list)): # initialization
    year_month = year_month_key_list[index]
    submission_year_month_dict[year_month] = {}
# for year_month in year_month_key_list: # initialization
#     submission_year_month_dict[year_month] = {}

In [32]:
ban_keywords = ['ban','sale', 'sales']
SF_keywords = ['san francisco', 'sf', 'sfo', 'san fran']

# Approach 1 Simple Combination
# ban_SF_keywords = []
# ban_SF_keywords.extend(ban_keywords)
# ban_SF_keywords.extend(SF_keywords)

# Approach 2 Cross Concatenation
ban_SF_keywords = [i+' '+j for i in ban_keywords for j in SF_keywords]
print('length of ban_SF_keywords: ', len(ban_SF_keywords))

length of ban_SF_keywords:  12


In [33]:
def getPushshiftData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+\
          str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)+'&sort=asc'
#     url = 'https://api.pushshift.io/reddit/search/submission/?title='+\
#           str(query)+'&size=1000'+'&subreddit='+str(sub)
#     print(url)
    r = requests.get(url)
    print(r)
    data = json.loads(r.text)
    return data['data']

In [34]:
def collect_submission(subm, year_month):
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
#     author_fullName = subm['author_fullname']
    sub_id = subm['id']
    score = subm['score']
    try:
        subreddit = subm['subreddit']
    except KeyError:
        subreddit = "None"
        
    try:
        self_text = subm['selftext']
    except KeyError:
        self_text = "None"
        
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']
    
    subData.append((sub_id, title, self_text, url, author, score,subreddit, created,numComms,permalink,flair))
#     subStats[sub_id] = subData
    submission_year_month_dict[year_month][sub_id] = subData
    

### Search keyword without specifying subreddit

In [35]:
month_second_pair_list

[(1459468800.0, 1461974400.0),
 (1462060800.0, 1464652800.0),
 (1464739200.0, 1467244800.0),
 (1467331200.0, 1469923200.0),
 (1470009600.0, 1472601600.0),
 (1472688000.0, 1475193600.0),
 (1475280000.0, 1477872000.0),
 (1477958400.0, 1480464000.0),
 (1480550400.0, 1483142400.0),
 (1483228800.0, 1485820800.0),
 (1485907200.0, 1488240000.0),
 (1488326400.0, 1490918400.0),
 (1491004800.0, 1493510400.0),
 (1493596800.0, 1496188800.0),
 (1496275200.0, 1498780800.0),
 (1498867200.0, 1501459200.0),
 (1501545600.0, 1504137600.0),
 (1504224000.0, 1506729600.0),
 (1506816000.0, 1509408000.0),
 (1509494400.0, 1512000000.0),
 (1512086400.0, 1514678400.0),
 (1514764800.0, 1517356800.0),
 (1517443200.0, 1519776000.0),
 (1519862400.0, 1522454400.0),
 (1522540800.0, 1525046400.0),
 (1525132800.0, 1527724800.0),
 (1527811200.0, 1530316800.0),
 (1530403200.0, 1532995200.0),
 (1533081600.0, 1535673600.0),
 (1535760000.0, 1538265600.0),
 (1538352000.0, 1540944000.0),
 (1541030400.0, 1543536000.0),
 (154362

In [37]:
subStats = {}
sub = "electronic_cigarette"

# go through each month
for index in range(len(month_second_pair_list)):
    after_start_month_second, before_end_month_second = month_second_pair_list[index]
    year_month = year_month_key_list[index]
    
    # go through search query
    for query in ban_SF_keywords:
        print('--------------------------------------')
        print('Query: ', query)
        after = str(after_start_month_second).split('.')[0]
        before = str(before_end_month_second).split('.')[0]



        data = getPushshiftData(query, after, before, sub)
        # Will run until all posts have been gathered from the 'after' date up until before date
        while len(data) > 0:
            for submission in data:
    #             print(submission)
    #             print(submission['title'])
    #             print(submission['subreddit'])
    #             print('-----------')

                collect_submission(submission, year_month)

            # Prepare for next iteration
            print(len(data))
            print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
            after = data[-1]['created_utc']
            data = getPushshiftData(query, after, before,sub)

        print(len(data))
#         print('length of submission_year_month_dict.keys(): ', len(submission_year_month_dict.keys()))
        time.sleep(.400)


--------------------------------------
Query:  ban san francisco
<Response [200]>
0
--------------------------------------
Query:  ban sf
<Response [200]>
0
--------------------------------------
Query:  ban sfo
<Response [200]>
0
--------------------------------------
Query:  ban san fran
<Response [200]>
0
--------------------------------------
Query:  sale san francisco
<Response [200]>
0
--------------------------------------
Query:  sale sf
<Response [200]>
0
--------------------------------------
Query:  sale sfo
<Response [200]>
0
--------------------------------------
Query:  sale san fran
<Response [200]>
0
--------------------------------------
Query:  sales san francisco
<Response [200]>
0
--------------------------------------
Query:  sales sf
<Response [200]>
0
--------------------------------------
Query:  sales sfo
<Response [200]>
0
--------------------------------------
Query:  sales san fran
<Response [200]>
0
--------------------------------------
Query:  ban san fra

<Response [200]>
0
--------------------------------------
Query:  sales sf
<Response [200]>
0
--------------------------------------
Query:  sales sfo
<Response [200]>
0
--------------------------------------
Query:  sales san fran
<Response [200]>
0
--------------------------------------
Query:  ban san francisco
<Response [200]>
0
--------------------------------------
Query:  ban sf
<Response [200]>
0
--------------------------------------
Query:  ban sfo
<Response [200]>
0
--------------------------------------
Query:  ban san fran
<Response [200]>
0
--------------------------------------
Query:  sale san francisco
<Response [200]>
0
--------------------------------------
Query:  sale sf
<Response [200]>
0
--------------------------------------
Query:  sale sfo
<Response [200]>
0
--------------------------------------
Query:  sale san fran
<Response [200]>
0
--------------------------------------
Query:  sales san francisco
<Response [200]>
0
--------------------------------------


<Response [200]>
3
2017-09-08 05:21:58
<Response [200]>
0
--------------------------------------
Query:  ban sf
<Response [200]>
1
2017-09-08 05:20:09
<Response [200]>
0
--------------------------------------
Query:  ban sfo
<Response [200]>
0
--------------------------------------
Query:  ban san fran
<Response [200]>
0
--------------------------------------
Query:  sale san francisco
<Response [200]>
0
--------------------------------------
Query:  sale sf
<Response [200]>
0
--------------------------------------
Query:  sale sfo
<Response [200]>
0
--------------------------------------
Query:  sale san fran
<Response [200]>
0
--------------------------------------
Query:  sales san francisco
<Response [200]>
0
--------------------------------------
Query:  sales sf
<Response [200]>
0
--------------------------------------
Query:  sales sfo
<Response [200]>
0
--------------------------------------
Query:  sales san fran
<Response [200]>
0
--------------------------------------
Query:

--------------------------------------
Query:  sale sfo
<Response [200]>
0
--------------------------------------
Query:  sale san fran
<Response [200]>
0
--------------------------------------
Query:  sales san francisco
<Response [200]>
0
--------------------------------------
Query:  sales sf
<Response [200]>
0
--------------------------------------
Query:  sales sfo
<Response [200]>
0
--------------------------------------
Query:  sales san fran
<Response [200]>
0
--------------------------------------
Query:  ban san francisco
<Response [200]>
3
2018-06-16 19:42:43
<Response [200]>
0
--------------------------------------
Query:  ban sf
<Response [200]>
3
2018-06-13 22:00:36
<Response [200]>
0
--------------------------------------
Query:  ban sfo
<Response [200]>
0
--------------------------------------
Query:  ban san fran
<Response [200]>
1
2018-06-14 07:49:55
<Response [200]>
0
--------------------------------------
Query:  sale san francisco
<Response [200]>
0
---------------

--------------------------------------
Query:  ban sf
<Response [200]>
0
--------------------------------------
Query:  ban sfo
<Response [200]>
0
--------------------------------------
Query:  ban san fran
<Response [200]>
0
--------------------------------------
Query:  sale san francisco
<Response [200]>
0
--------------------------------------
Query:  sale sf
<Response [200]>
0
--------------------------------------
Query:  sale sfo
<Response [200]>
0
--------------------------------------
Query:  sale san fran
<Response [200]>
0
--------------------------------------
Query:  sales san francisco
<Response [200]>
0
--------------------------------------
Query:  sales sf
<Response [200]>
0
--------------------------------------
Query:  sales sfo
<Response [200]>
0
--------------------------------------
Query:  sales san fran
<Response [200]>
0
--------------------------------------
Query:  ban san francisco
<Response [200]>
5
2019-03-26 12:16:44
<Response [200]>
0
-------------------

In [38]:
submission_year_month_dict

{'2016-04': {},
 '2016-05': {},
 '2016-06': {},
 '2016-07': {},
 '2016-08': {},
 '2016-09': {},
 '2016-10': {},
 '2016-11': {},
 '2016-12': {},
 '2017-01': {},
 '2017-02': {},
 '2017-03': {},
 '2017-04': {'66cjtg': [('66cjtg',
    'San Francisco Board of supervisors propose ban of all flavored cigarettes including vape juice with Nicotine',
    "I know laws like this have been proposed in other cities but the problem is it'll probably pass here in SF. I guess they just want you to keep smoking (non-menthol). ",
    'https://www.reddit.com/r/electronic_cigarette/comments/66cjtg/san_francisco_board_of_supervisors_propose_ban_of/',
    'vapebeau',
    1,
    'electronic_cigarette',
    datetime.datetime(2017, 4, 19, 19, 37, 23),
    4,
    '/r/electronic_cigarette/comments/66cjtg/san_francisco_board_of_supervisors_propose_ban_of/',
    'Advocacy')]},
 '2017-05': {'6bkc0l': [('6bkc0l',
    'San Francisco Flavor Ban hearing on the 24th',
    "Unfortunately, San Francisco, being a major city

In [39]:
for year_month in submission_year_month_dict:
    print(year_month + ":", len(submission_year_month_dict[year_month]))
    
# print('length of subStats.keys(): ', len(subStats.keys()))
# subStats.keys()

2016-04: 0
2016-05: 0
2016-06: 0
2016-07: 0
2016-08: 0
2016-09: 0
2016-10: 0
2016-11: 0
2016-12: 0
2017-01: 0
2017-02: 0
2017-03: 0
2017-04: 1
2017-05: 1
2017-06: 10
2017-07: 3
2017-08: 2
2017-09: 3
2017-10: 0
2017-11: 0
2017-12: 0
2018-01: 1
2018-02: 0
2018-03: 1
2018-04: 1
2018-05: 3
2018-06: 8
2018-07: 0
2018-08: 0
2018-09: 0
2018-10: 0
2018-11: 0
2018-12: 0
2019-01: 0
2019-02: 0
2019-03: 6
2019-04: 5


In [40]:
submission_year_month_dict['2017-06']

{'6iq0kc': [('6iq0kc',
   'San Francisco votes to ban sales of vaping flavored liquid',
   '',
   'http://m.sfgate.com/news/medical/article/San-Francisco-to-ban-sales-of-vaping-flavored-11237149.php',
   'catsx3',
   0,
   'electronic_cigarette',
   datetime.datetime(2017, 6, 22, 0, 34, 2),
   3,
   '/r/electronic_cigarette/comments/6iq0kc/san_francisco_votes_to_ban_sales_of_vaping/',
   'NaN')],
 '6itwc5': [('6itwc5',
   'San Francisco to ban sales of vaping flavored liquid',
   '[deleted]',
   'https://www.google.ca/url?sa=t&amp;source=web&amp;rct=j&amp;url=/amp/abcnews.go.com/amp/Health/wireStory/san-francisco-ban-sales-vaping-flavored-liquid-48192121&amp;ved=0ahUKEwj22eeX19HUAhUJxoMKHccRAWUQiJQBCC4wAQ&amp;usg=AFQjCNEZf6MWE6a6X2B68X1fZRLHkT13uw',
   '[deleted]',
   0,
   'electronic_cigarette',
   datetime.datetime(2017, 6, 22, 14, 40, 1),
   3,
   '/r/electronic_cigarette/comments/6itwc5/san_francisco_to_ban_sales_of_vaping_flavored/',
   'NaN')],
 '6j9x4e': [('6j9x4e',
   'A littl

In [22]:
def save_submission_year_month_dict_to_file(fileName, submission_year_month_dict):
    location = "/mnt/volume-second-5T/subreddits/"
    
    for year_month in submission_year_month_dict.keys():
        length_count = 0
        temp_fileName = fileName + "_" + year_month + ".csv"
        print(temp_fileName)
        file = location + temp_fileName
        with open(file, 'w', newline='', encoding='utf-8') as file: 
            a = csv.writer(file, delimiter=',')
            headers = ["Post_ID","Title","Self_text", "Url","Author","Score","Subreddit","Publish_date","Number_of_comments","Permalink","Flair"]
            a.writerow(headers)
            for sub in submission_year_month_dict[year_month]:
                a.writerow(submission_year_month_dict[year_month][sub][0])
                length_count+=1

            print(str(length_count) + " submissions have been saved")
        
save_submission_year_month_dict_to_file("electronic_cigarette_ban_and_SF_keywords_collection", submission_year_month_dict)

electronic_cigarette_ban_and_SF_keywords_collection_2016-04.csv
11 submissions have been saved
electronic_cigarette_ban_and_SF_keywords_collection_2016-05.csv
19 submissions have been saved
electronic_cigarette_ban_and_SF_keywords_collection_2016-06.csv
7 submissions have been saved
electronic_cigarette_ban_and_SF_keywords_collection_2016-07.csv
8 submissions have been saved
electronic_cigarette_ban_and_SF_keywords_collection_2016-08.csv
6 submissions have been saved
electronic_cigarette_ban_and_SF_keywords_collection_2016-09.csv
7 submissions have been saved
electronic_cigarette_ban_and_SF_keywords_collection_2016-10.csv
4 submissions have been saved
electronic_cigarette_ban_and_SF_keywords_collection_2016-11.csv
1 submissions have been saved
electronic_cigarette_ban_and_SF_keywords_collection_2016-12.csv
3 submissions have been saved
electronic_cigarette_ban_and_SF_keywords_collection_2017-01.csv
6 submissions have been saved
electronic_cigarette_ban_and_SF_keywords_collection_2017-0